# LA Dodgers Standings, 1958-2023
> This notebook downloads historic standing tables from [Baseball Reference](https://www.baseball-reference.com/teams/LAD/2024-schedule-scores.shtml) and outputs them to CSV, JSON and Parquet formats for later analysis and visualization.

---

#### Import Python tools and Jupyter config

In [2]:
import pandas as pd
import jupyter_black
from time import sleep
from tqdm.notebook import tqdm

In [3]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

## Fetch

#### List comprehension of historic urls

In [4]:
urls = [
    f"https://www.baseball-reference.com/teams/LAD/{year}-schedule-scores.shtml"
    for year in range(1958, 2024)
]

#### Loop through urls, fetch standings table, store in list of dataframes

In [5]:
dfs = []

for url in tqdm(urls):
    year = url.split("/")[5].replace("-schedule-scores.shtml", "")
    src_df = (
        pd.read_html(url)[0]
        .query("Tm !='Tm' and Inn != 'Game Preview, and Matchups'")
        .drop(["Unnamed: 2", "Streak", "Orig. Scheduled"], axis=1)
        .rename(columns={"Unnamed: 4": "home_away"})
        .assign(season=year)
    )
    dfs.append(src_df)
    sleep(4)

  0%|          | 0/66 [00:00<?, ?it/s]

#### Concatenate into one historic dataframe

In [195]:
src = pd.concat(dfs)

---

## Process

#### Clean columns

In [196]:
src.columns = src.columns.str.lower().str.replace("/", "_").str.replace("-", "-")

In [197]:
src.columns = [
    "gm",
    "date",
    "tm",
    "home_away",
    "opp",
    "result",
    "r",
    "ra",
    "inn",
    "record",
    "rank",
    "gb",
    "win",
    "loss",
    "save",
    "time",
    "day_night",
    "attendance",
    "cli",
    "year",
]

#### Split, format date

In [198]:
src[["weekday", "date"]] = src["date"].str.split(", ", expand=True)

In [199]:
src["date"] = src["date"].str.replace(" (1)", "").str.replace(" (2)", "")

In [200]:
src["game_date"] = pd.to_datetime(src["date"] + ", " + src["year"], format="%b %d, %Y")

#### Clean home-away column

In [201]:
src.loc[src.home_away == "@", "home_away"] = "away"
src.loc[src.home_away.isna(), "home_away"] = "home"

#### Format "games back" as a number (positive = lead; negative = behind)

In [202]:
src["gb"] = (
    src["gb"].str.replace("up ", "up").str.replace("up", "+").str.replace("Tied", "0")
)

In [203]:
src["gb"] = src["gb"].apply(
    lambda x: float(x) if x.startswith("+") else -float(x) if float(x) != 0 else 0
)

#### The *number* of games

In [204]:
src["attendance"] = src["attendance"].fillna(0)
src["gm"] = src["gm"].astype(int)
src[["r", "ra", "attendance", "gm"]] = src[["r", "ra", "attendance", "gm"]].astype(int)

#### Convert the 'time' column to timedelta, then to minutes

In [205]:
src["time"] = src["time"] + ":00"

In [206]:
src["time_minutes"] = pd.to_timedelta(src["time"]).dt.total_seconds() / 60
src["time_minutes"] = src["time_minutes"].astype(int)

#### Just the columns we need, in a clean dataframe

In [207]:
df = src[
    [
        "gm",
        "game_date",
        "home_away",
        "opp",
        "result",
        "r",
        "ra",
        "record",
        "rank",
        "gb",
        "time",
        "time_minutes",
        "day_night",
        "attendance",
        "year",
    ]
].copy()

In [208]:
df.dtypes

gm                       int64
game_date       datetime64[ns]
home_away               object
opp                     object
result                  object
r                        int64
ra                       int64
record                  object
rank                    object
gb                     float64
time                    object
time_minutes             int64
day_night               object
attendance               int64
year                    object
dtype: object

---

## Exports

#### CSV format

In [209]:
df.to_csv("../data/processed/dodgers_standings_1958_2023.csv", index=False)

#### JSON

In [210]:
df.to_json(
    "../data/processed/dodgers_standings_1958_2023.json", indent=4, orient="records"
)

#### Parquet

In [211]:
df.to_parquet("../data/processed/dodgers_standings_1958_2023.parquet", index=False)

In [212]:
df.columns

Index(['gm', 'game_date', 'home_away', 'opp', 'result', 'r', 'ra', 'record',
       'rank', 'gb', 'time', 'time_minutes', 'day_night', 'attendance',
       'year'],
      dtype='object')

In [213]:
table_description = df.dtypes.reset_index()
table_description["column_description"] = "Column description here"
table_description.columns = ["column_name", "column_type", "column_description"]

In [214]:
df.head()

,gm,game_date,home_away,opp,result,r,ra,record,rank,gb,time,time_minutes,day_night,attendance,year
0,1,1958-04-15,away,SFG,L,0,8,0-1,5,-1.0,2:29:00,149,D,23448,1958
1,2,1958-04-16,away,SFG,W,13,1,1-1,4,-0.5,3:03:00,183,N,22735,1958
2,3,1958-04-17,away,SFG,L,4,7,1-2,6,-1.5,2:50:00,170,D,12520,1958
3,4,1958-04-18,home,SFG,W,6,5,2-2,3,-1.5,3:00:00,180,D,78672,1958
4,5,1958-04-19,home,SFG,L,4,11,2-3,5,-2.5,2:37:00,157,D,41303,1958
